<a href="https://colab.research.google.com/github/MalakhovDenis/DLS-DL2/blob/main/%5Bhomework1%5Dsimple_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=300>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin

--2025-03-03 13:59:52--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-03-03 13:59:53--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin’

SO_vectors_200.bin   68%[============>       ] 951.77M   691KB/s    in 24m 59s 

2025-03-03 14:24:52 (650 KB/s) - Connection closed at byte 998000408. Retrying.

--2025-03-03 14:24:53--  (try: 2)  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response...

In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


Найдем наиболее близкие слова к слову `dog`:

#### ***❓Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [4]:
word_to_search = 'cat'
topn = 5

try:
    # Получаем список наиболее похожих слов
    similar_words = wv_embeddings.most_similar(positive='dog', topn=topn)

    # Проверяем, есть ли искомое слово в списке
    for position, (word, _) in enumerate(similar_words, start=1):
        if word == word_to_search:
            print(f"'{word_to_search}' входит в топ-{topn} ближайших слов к 'dog' на позиции {position}.")
            break
    else:
        print(f"'{word_to_search}' не входит в топ-{topn} ближайших слов к 'dog'.")

except KeyError:
    print("Слово 'dog' не найдено в векторных представлениях.")


'cat' не входит в топ-5 ближайших слов к 'dog'.


#### ***✅Ответ на Вопрос 1:***
`cat` не входит в топ-5 ближайших слов к `dog`

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [5]:
import numpy as np
import re

# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)

In [6]:
tokenizer = MyTokenizer()

In [7]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    text_tokenized = tokenizer.tokenize(text=question.lower())
    return_embeddings = np.zeros(shape=dim)
    n = 0
    for word in text_tokenized:
      if word in embeddings:
        return_embeddings += embeddings[word]
        n += 1
    if n > 0:
      return return_embeddings / n
    else:
      return return_embeddings

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***❓Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [8]:
question = "I love neural networks"
index = 2
print(f"Третья компонента вектора: {round(question_to_vec(question=question, embeddings=wv_embeddings, tokenizer=tokenizer, dim=200)[index], 2)}")

Третья компонента вектора: -1.29


#### ***✅Ответ на Вопрос 2:***

Третья (с индексом 2) компонента вектора предложения `I love neural networks` это `-1.29`

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=400>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***❓Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)



In [9]:
print(round(1/(np.log2(1+9)),2))

0.3


#### ***✅Ответ на Вопрос 3:***


Для вычисления `DCG@10`, когда $rank\_q_i^{'} = 9$, используем формулу:

$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$

В нашем случае ` N = 1 ` (так как у нас только один вопрос), ` K = 10 `, и $rank\_q_i^{'} = 9$

Теперь подставим значения в формулу:

Проверяем условие $rank\_q_i^{'} = 9 \leq K$:

$9 \leq 10$ — условие верно, значит $[rank\_q_i^{'} \leq 10] = 1$

Вычисляем $\frac{1}{\log_2(1+rank\_q_i^{'})}$:

$\log_2(1 + 9) = \log_2(10)$

$\log_2(10) \approx 3.32193$


Теперь подставим это значение в формулу:

$\text{DCG@10} = \frac{1}{1} \cdot \frac{1}{\log_2(10)} \cdot 1 = \frac{1}{3.32193} \approx 0.3010$

Округляем до одного знака после запятой:

$ \text{DCG@10} \approx 0.3 $

**Таким образом, ответ: 0.3**

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***❓Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?



#### ***✅Ответ на Вопрос 4:***

`Hits@47:` `1`, если ранг всех дубликатов $\leq K$ (в нашем случае 47)

`DCG@1:` `0`, если ранг всех дубликатов $> K$ (в нашем случае 1)

**Таким образом, максимум разницы `Hits@47 - DCG@1` может быть равен `1` в случае, когда ранги всех дубликатов находятся в интервале $>1$ и $\leq 47$**

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [10]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть Hits@k
    """
    # Подсчитываем количество дубликатов, чей ранг <= k
    n = 0
    for i in dup_ranks:
      if i <= k:
        n += 1
    hits_value = n / len(dup_ranks)
    return hits_value

In [11]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [12]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        k: пороговое значение для ранга
        result: вернуть DCG@k
    """
    # Вычисляем сумму для всех релевантных дубликатов
    dcg = 0
    for i in dup_ranks:
      if i <= k:
        dcg += 1 / np.log2(1 + i)
    # Делим на общее количество вопросов
    dcg_value = dcg / len(dup_ranks)
    return dcg_value

In [13]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [14]:
import pandas as pd

In [15]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dict_ranking = {candidate: rank + 1 for rank, candidate in enumerate(candidates_ranking[0])}
dup_ranks = [dict_ranking[copy_answers[0]]]

# вычисляем метрику для разных k
# print('Ранг дубликата:', dup_ranks)
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [16]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [17]:
!gdown 1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_

Downloading...
From (original): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_
From (redirected): https://drive.google.com/uc?id=1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_&confirm=t&uuid=a27b696a-ee2f-481e-b004-5a003ab4c74c
To: /content/stackoverflow_similar_questions.zip
100% 131M/131M [00:03<00:00, 41.9MB/s]


In [18]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [19]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [20]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [21]:
len(validation_data)

3760

Размер нескольких первых строк

In [22]:
for i in range(25):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001
6 1001
7 1001
8 1001
9 1001
10 1001
11 1001
12 1001
13 1001
14 1001
15 1001
16 1001
17 1001
18 1001
19 1001
20 1001
21 1001
22 1001
23 1001
24 1001
25 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [24]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    question_emb = question_to_vec(question, embeddings, tokenizer, dim).reshape(1,-1)
    candidates_emb = [question_to_vec(c, embeddings, tokenizer, dim) for c in candidates]
    cosine_s = pd.Series(cosine_similarity(question_emb, candidates_emb).squeeze()).sort_values()
    result = [(i, candidates[i]) for i in cosine_s.index][::-1]

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [25]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [26]:
results = list()
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()
        results.append(ranks)

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



In [27]:
results

[[(1, 'C# create cookie from string and send it'),
  (0, 'Convert Google results object (pure js) to Python object'),
  (2, 'How to use jQuery AJAX for an outside domain?')],
 [(0, 'Getting all list items of an unordered list in PHP'),
  (2, 'select2 not displaying search results'),
  (1, 'WPF- How to update the changes in list item of a list')]]

Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [28]:
"""# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт"""

"# должно вывести\nresults = [[(1, 'C# create cookie from string and send it'),\n            (0, 'Convert Google results object (pure js) to Python object'),\n            (2, 'How to use jQuery AJAX for an outside domain?')],\n           [(*, 'Getting all list items of an unordered list in PHP'), #скрыт\n            (*, 'select2 not displaying search results'), #скрыт\n            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт"

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***❓Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


#### ***✅Ответ на Вопрос 5:***
021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [29]:
from tqdm.notebook import tqdm

In [30]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [31]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.415 | Hits@   1: 0.415
DCG@   5: 0.502 | Hits@   5: 0.582
DCG@  10: 0.524 | Hits@  10: 0.650
DCG@ 100: 0.570 | Hits@ 100: 0.874
DCG@ 500: 0.583 | Hits@ 500: 0.973
DCG@1000: 0.586 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [32]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [33]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [34]:
# Создаем общий корпус текстов
corpus = [list(tokenizer.tokenize(" ".join(corp))) for corp in train_data]

In [35]:
corpus[0]

['converting',
 'string',
 'to',
 'list',
 'Convert',
 'Google',
 'results',
 'object',
 'pure',
 'js',
 'to',
 'Python',
 'object']

In [36]:
from gensim.models import Word2Vec
embeddings_trained = Word2Vec(
    sentences=corpus,        # Корпус токенизированных текстов
    vector_size=200,         # Размерность векторов
    window=3,                # Размер окна контекста
    min_count=5,             # Минимальная частота слов
    workers=4                # Количество потоков
).wv

In [37]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [38]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.246 | Hits@   1: 0.246
DCG@   5: 0.313 | Hits@   5: 0.374
DCG@  10: 0.336 | Hits@  10: 0.443
DCG@ 100: 0.387 | Hits@ 100: 0.697
DCG@ 500: 0.414 | Hits@ 500: 0.907
DCG@1000: 0.424 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

🤔 Перед тем как сделать окончательные выводы о полученных результатах, я считаю целесообразным провести дополнительные эксперименты, например, с использованием библиотеки NLTK. Также планирую разработать собственный предобработчик данных.



### ***🔬Эксперимент с библиотекой NLTK***

В процессе проведения эксперимента я планирую выполнить несколько ключевых этапов предобработки текстовых данных.

Первым шагом будет преобразование всего текста в нижний регистр. Это необходимо для того, чтобы устранить различия между словами, которые могут возникнуть из-за использования заглавных и строчных букв. Например, слова "Text" и "text" будут восприниматься как одно и то же слово, что поможет улучшить качество дальнейшего анализа.

Следующим этапом станет токенизация с использованием библиотеки NLTK. Токенизация — это процесс разделения текста на отдельные элементы, или токены, такие как слова и знаки препинания. Это позволит мне более эффективно работать с текстом, анализируя его на уровне отдельных слов. Например, предложение "This is a sample sentence." будет разбито на токены: ["This", "is", "a", "sample", "sentence", "."].

После токенизации я проведу лемматизацию, также с помощью NLTK. Лемматизация — это процесс приведения слов к их базовой форме, или лемме. Например, слова "running", "ran" и "runner" будут приведены к одной форме "run". Это поможет сократить количество уникальных слов в анализируемом тексте и улучшить качество модели.

Наконец, я удалю стоп-слова на английском языке, используя инструменты NLTK. Стоп-слова — это распространенные слова, такие как "and", "in", "on", которые не несут значимой информации для анализа. Удаление этих слов позволит сосредоточиться на более значимых терминах и улучшить результаты анализа.

Таким образом, последовательное выполнение этих шагов предобработки поможет мне подготовить данные для более глубокого анализа и получения более точных результатов.


In [47]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import WordNetLemmatizer

# Скачиваем необходимые ресурсы NLTK
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")
nltk.download("punkt_tab")

# Класс для токенизации текста
class CustomTokenizer:
    def __init__(self):
        pass

    def tokenize(self, text=None, lemmatizer=None, stopwords=None):
        if text is None:
            return []

        # Преобразуем текст в нижний регистр и токенизируем
        tokens = word_tokenize(text.lower())

        # Если есть лемматизатор, применяем его
        if lemmatizer:
            tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Убираем стоп-слова, если они есть
        if stopwords is not None:
            tokens = [word for word in tokens if word not in stopwords]

        return tokens

# Функция для преобразования вопроса в вектор
def question_to_vector(question=None, embeddings=None, tokenizer=None, lemmatizer=None, stopwords=None, vector_dim=200):
    # Создаем вектор нулей
    question_vector = np.zeros(shape=vector_dim)
    token_count = 0

    # Токенизируем вопрос и проходим по токенам
    for token in tokenizer.tokenize(text=question, lemmatizer=lemmatizer, stopwords=stopwords):
        # Если токен есть в эмбеддингах, добавляем его в вектор
        if token in embeddings:
            question_vector += embeddings[token]
            token_count += 1

    # Если были токены, нормализуем вектор
    if token_count > 0:
        return question_vector / token_count
    else:
        return question_vector  # Возвращаем нулевой вектор, если токенов нет

# Функция для ранжирования кандидатов по схожести с вопросом
def rank_candidates(question=None, candidates=None, embeddings=None, tokenizer=None, lemmatizer=None, stopwords=None, vector_dim=200):
    # Преобразуем вопрос в вектор
    question_vector = question_to_vector(question, embeddings, tokenizer, lemmatizer, stopwords, vector_dim).reshape(1, -1)

    # Преобразуем кандидатов в векторы
    candidate_vectors = []
    for candidate in candidates:
        candidate_vector = question_to_vector(candidate, embeddings, tokenizer, lemmatizer, stopwords, vector_dim)
        candidate_vectors.append(candidate_vector)

    # Вычисляем косинусное сходство
    cosine_similarities = pd.Series(cosine_similarity(question_vector, candidate_vectors).squeeze()).sort_values(ascending=False)

    # Возвращаем индексы и строки кандидатов, отсортированные по схожести
    return [(index, candidates[index]) for index in cosine_similarities.index]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [48]:
tokenizer_nltk = CustomTokenizer()
lemmatizator_nltk = WordNetLemmatizer()
stopwords_nltk = set(stopwords.words('english'))

In [49]:
# Инициализация токенизатора и стоп-слов
tokenizer_custom = CustomTokenizer()  # Создаем экземпляр нашего токенизатора
stopwords_nltk = set(stopwords.words('english'))  # Получаем набор английских стоп-слов

# Список для хранения рангов
wv_ranking = []
max_validation_examples = 1000  # Максимальное количество примеров для проверки

# Проходим по данным валидации
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:  # Если достигли максимального количества примеров, выходим из цикла
        break
    q, *ex = line  # Извлекаем вопрос и кандидатов из строки
    # Ранжируем кандидатов по схожести с вопросом
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer_custom, None, stopwords_nltk)
    # Добавляем индекс правильного ответа в список рангов
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

# Вывод результатов
# Проверяем различные значения k для оценки
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    dcg_value = dcg_score(wv_ranking, k)  # Вычисляем DCG
    hits_value = hits_count(wv_ranking, k)  # Вычисляем количество попаданий
    # Печатаем результаты
    print(f"DCG@{k:4d}: {dcg_value:.3f} | Hits@{k:4d}: {hits_value:.3f}")

100%|██████████| 6/6 [00:00<00:00, 399.53it/s]

DCG@   1: 0.401 | Hits@   1: 0.401
DCG@   5: 0.487 | Hits@   5: 0.566
DCG@  10: 0.509 | Hits@  10: 0.634
DCG@ 100: 0.555 | Hits@ 100: 0.859
DCG@ 500: 0.570 | Hits@ 500: 0.969
DCG@1000: 0.573 | Hits@1000: 1.000


In [50]:
# Инициализация токенизатора, лемматизатора и стоп-слов
tokenizer_custom = CustomTokenizer()  # Создаем экземпляр нашего токенизатора
lemmatizer_nltk = WordNetLemmatizer()  # Создаем экземпляр лемматизатора
stopwords_nltk = set(stopwords.words('english'))  # Получаем набор английских стоп-слов

# Список для хранения рангов
wv_ranking = []
max_validation_examples = 1000  # Максимальное количество примеров для проверки

# Проходим по данным валидации
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:  # Если достигли максимального количества примеров, выходим из цикла
        break
    q, *ex = line  # Извлекаем вопрос и кандидатов из строки
    # Ранжируем кандидатов по схожести с вопросом
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer_custom, lemmatizer_nltk, stopwords_nltk)
    # Добавляем индекс правильного ответа в список рангов
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

# Настройка формата вывода для numpy
np.set_printoptions(precision=3, suppress=True)  # Устанавливаем формат вывода для numpy

# Вывод результатов
# Проверяем различные значения k для оценки
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    dcg_value = dcg_score(wv_ranking, k)  # Вычисляем DCG
    hits_value = hits_count(wv_ranking, k)  # Вычисляем количество попаданий
    # Печатаем результаты
    print(f"DCG@{k:4d}: {dcg_value:.3f} | Hits@{k:4d}: {hits_value:.3f}")

100%|██████████| 6/6 [00:00<00:00, 239.17it/s]

DCG@   1: 0.391 | Hits@   1: 0.391
DCG@   5: 0.480 | Hits@   5: 0.562
DCG@  10: 0.501 | Hits@  10: 0.628
DCG@ 100: 0.547 | Hits@ 100: 0.855
DCG@ 500: 0.562 | Hits@ 500: 0.964
DCG@1000: 0.565 | Hits@1000: 1.000


###🔬Эксперимент с самостоятельной предобработкой


В процессе проведения эксперимента я выполню несколько аналогичных ключевых этапов предобработки текстовых данных, но теперь с использованием собственного предобработчика.

In [51]:
import numpy as np
import re
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

class Tokenizer:
    def __init__(self):
        pass

    def tokenize(self, text=None, lemmatizer=None, stop_words=None):
        # Создаем регулярное выражение для стоп-слов
        stop_words_regex = r'\b|\b'.join(stop_words)

        # Приводим текст к нижнему регистру и удаляем все символы, кроме букв и пробелов
        cleaned_text = re.sub('[^А-Яа-яЁёA-Za-z\s]', ' ', text.lower())

        # Если есть лемматизатор, применяем его к тексту
        if lemmatizer:
            cleaned_text = " ".join([word.lemma_ for word in lemmatizer(cleaned_text)])

        # Удаляем стоп-слова из текста
        cleaned_text = re.sub(stop_words_regex, ' ', cleaned_text)

        # Удаляем лишние пробелы
        cleaned_text = re.sub('\s\s+', ' ', cleaned_text)

        # Возвращаем список токенов
        return cleaned_text.split()

def question_to_vector(question=None, embeddings=None, tokenizer=None, lemmatizer=None, stop_words=None, dim=200):
    """
        question: строка
        embeddings: наше векторное представление слов
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """
    question_vector = np.zeros(shape=dim)
    word_count = 0

    # Токенизируем вопрос и проходим по каждому слову
    for word in tokenizer.tokenize(text=question, lemmatizer=lemmatizer, stop_words=stop_words):
        if word in embeddings:
            question_vector += embeddings[word]
            word_count += 1

    # Если были найдены слова, возвращаем средний вектор
    if word_count > 0:
        return question_vector / word_count
    else:
        return question_vector

def rank_candidates(question, candidates, embeddings, tokenizer, lemmatizer, stop_words, dim=200):
    """
        question: строка
        candidates: массив строк (кандидатов) [a, b, c]
        return: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    # Получаем вектор для вопроса
    question_vector = question_to_vector(question, embeddings, tokenizer, lemmatizer, stop_words, dim).reshape(1, -1)

    # Получаем векторы для всех кандидатов
    candidate_vectors = [question_to_vector(candidate, embeddings, tokenizer, lemmatizer, stop_words, dim) for candidate in candidates]

    # Вычисляем косинусное сходство между вектором вопроса и векторами кандидатов
    cosine_similarities = pd.Series(cosine_similarity(question_vector, candidate_vectors).squeeze()).sort_values()

    # Возвращаем отсортированные пары (индекс, кандидат)
    return [(i, candidates[i]) for i in cosine_similarities.index][::-1]

In [52]:
# Список стоп-символов
stop_letters = ['at', 'q', 'w', 'e', 'r', 't', 'y', 'u', 'i', 'o', 'p',
                'a', 's', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'z', 'x',
                'c', 'v', 'b', 'n', 'm']

# Объединяем стоп-слова из NLTK и наши стоп-символы
stopwords_combined = list(set(list(stopwords_nltk) + stop_letters))

In [53]:
import spacy

# Инициализируем токенизатор и лемматизатор
tokenizer = Tokenizer()
lemmatizer = spacy.load('en_core_web_sm')

In [54]:
# Список для хранения рангов
wv_ranking = []
max_validation_examples = 1000

# Проходим по данным валидации
for index, line in enumerate(tqdm(validation_data)):
    if index == max_validation_examples:
        break
    question, *examples = line
    ranks = rank_candidates(question, examples, wv_embeddings, tokenizer, None, stopwords_combined)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

# Вычисляем и выводим метрики DCG и Hits для различных значений k
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))


100%|██████████| 6/6 [00:00<00:00, 407.10it/s]

DCG@   1: 0.424 | Hits@   1: 0.424
DCG@   5: 0.515 | Hits@   5: 0.598
DCG@  10: 0.534 | Hits@  10: 0.659
DCG@ 100: 0.579 | Hits@ 100: 0.876
DCG@ 500: 0.592 | Hits@ 500: 0.975
DCG@1000: 0.595 | Hits@1000: 1.000


In [55]:
# Сбросим список рангов для следующего расчета
wv_ranking = []
max_validation_examples = 1000

# Проходим по данным валидации снова
for index, line in enumerate(tqdm(validation_data)):
    if index == max_validation_examples:
        break
    question, *examples = line
    ranks = rank_candidates(question, examples, wv_embeddings, tokenizer, lemmatizer, stopwords_combined)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

# Вычисляем и выводим метрики DCG и Hits для различных значений k
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))


100%|██████████| 6/6 [00:00<00:00, 400.74it/s]

DCG@   1: 0.397 | Hits@   1: 0.397
DCG@   5: 0.488 | Hits@   5: 0.569
DCG@  10: 0.515 | Hits@  10: 0.652
DCG@ 100: 0.559 | Hits@ 100: 0.867
DCG@ 500: 0.573 | Hits@ 500: 0.971
DCG@1000: 0.576 | Hits@1000: 1.000


## Вывод:



***Какой принцип токенизации даёт качество лучше и почему?***


Лучшее качество токенизации достигается с использованием специализированных токенизаторов, таких как MyTokenizer. Этот токенизатор был разработан с учетом особенностей конкретного корпуса текстов, что позволяет ему более точно разделять текст на значимые единицы. Важно также отметить, что удаление стоп-слов в процессе токенизации значительно улучшает метрики, так как это позволяет сосредоточиться на более значимых словах, которые несут смысловую нагрузку.

***Помогает ли нормализация слов?***


Нормализация слов, включая стемминг и лемматизацию, в данном контексте, как правило, снижает качество метрик. Сохранение оригинальной формы слов может быть более полезным для анализа, так как это позволяет лучше учитывать контекст и семантику, что особенно важно в технических текстах, где точность формулировок имеет значение.

***Какие эмбеддинги лучше справляются с задачей и почему?***


Предобученные эмбеддинги показывают лучшие результаты, так как они обучены на больших корпусах текстов, что позволяет им лучше захватывать семантические связи и контексты слов. Эти модели обеспечивают более глубокое понимание языка и способны учитывать разнообразие форм слов, что делает их более эффективными в задачах обработки естественного языка.

***Почему получилось плохое качество решения задачи?***


Плохое качество решения задачи может быть связано с недостаточной предобработкой текста, выбором неподходящих эмбеддингов или использованием простых моделей, таких как word2vec, которые не учитывают контекст и семантику. Также, если в обучающей выборке присутствуют не все пары вопросов, это может негативно сказаться на качестве модели.

***Предложите свой подход к решению задачи.***


Рекомендуется использовать собственные токенизаторы с удалением стоп-слов, что позволит улучшить качество анализа. Также стоит рассмотреть возможность дообучения предобученных эмбеддингов на текущем корпусе текстов, чтобы адаптировать их к специфике задачи. В дополнение, использование более сложных моделей, таких как рекуррентные нейронные сети (RNN) или трансформеры, может значительно повысить качество решения задачи.